In [ ]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')

%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"
!ls | wc -l

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import xgboost as xgb

In [ ]:
# Load Dataset
df = pd.read_csv('df.csv')

In [ ]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [ ]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [ ]:
# without demographics
#df = df.iloc[::,:74]

In [ ]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [ ]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [ ]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [ ]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [ ]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [ ]:
file_name = 'XGB.60d.1d'
days = 5
num_companies = 29  # 29 companies in our dataset.
step_rows  = 24 * num_companies * days # 24 time periods per day per stock
total_rows = len(df['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2022-06-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model = XGBRegressor(n_estimators=300, max_depth=100, learning_rate=0.1, objective='reg:squarederror'
                     , alpha=10, tree_method="gpu_hist")

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [ ]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [ ]:
result[25::29]

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
25,2022-09-01 09:30:00,274.66,258.488373,16.171627,10.925955
54,2022-09-01 09:45:00,273.62,257.267090,16.352910,10.925955
83,2022-09-01 10:00:00,271.36,258.107330,13.252670,10.925955
112,2022-09-01 10:15:00,273.14,256.628998,16.511002,10.925955
141,2022-09-01 10:30:00,269.54,255.703949,13.836051,10.925955
...,...,...,...,...,...
56256,2022-12-30 14:15:00,121.46,113.230789,8.229211,11.307637
56285,2022-12-30 14:30:00,121.30,113.247269,8.052731,11.307637
56314,2022-12-30 14:45:00,121.51,114.757744,6.752256,11.307637
56343,2022-12-30 15:00:00,122.42,114.972076,7.447924,11.307637


In [ ]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [ ]:
predicted, actual,duration=result['PREDICTED'],result['ACTUAL'],result['TRAIN_DURATION']

In [ ]:
rmse = np.sqrt(mean_squared_error(result['ACTUAL'], result['PREDICTED']))
# Calculate MAPE %
mape = mean_absolute_percentage_error(result['ACTUAL'], result['PREDICTED']) * 100
# Calculate MPE %
mpe = mean_positive_error(result['ACTUAL'], result['PREDICTED'])
# Calculate MTT in seconds
mtt =duration.sum()

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mtt], index=['	stock_ID','MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
rmse,mape,mpe,metrics_df['MTT'].sum()

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mape, mpe, mtt], index=['stock_ID',	'MAPE',	'MPE',	'MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
#metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,4.027591,2.275192,2.328860,11.018598
1,AMD,3.836898,4.659402,2.234882,11.018598
2,AMZN,6.189824,4.311382,3.112893,11.018598
3,BA,9.346960,4.377279,1.668990,11.018598
4,BABA,6.964776,6.640908,2.432106,11.018598
5,BAC,1.394197,3.154781,0.497526,11.018598
6,C,1.207286,2.157968,0.526565,11.018598
7,CSCO,1.162816,1.968814,0.303394,11.018598
8,CVX,6.224248,3.015460,1.496944,11.018598
9,DIS,4.696966,3.257843,2.233706,11.018598


In [ ]:
avg_values=metrics_df.mean()
avg_df = pd.DataFrame(avg_values).T
avg_df